### 0. Setup

In [2]:
import openai
import pandas as pd
from io import StringIO
from typing import List
from secret_keys import OPENAI_API_KEY
openai.api_key = OPENAI_API_KEY
from utils import complete_prompt
from pandas.io import json as json_pd
PSINJECT = "data/sample_logs/windows_privesc_empire_psinject.json"
MIMIKATZ = "data/sample_logs/mimikatz_CVE-2020-1472_Unauthenticated_NetrServerAuthenticate2_2020-09-16233923.json"
SYSTEM_ROLE = "You are a skilled cybersecurity incident responder hunting for threats."

# 1. Detect signs of windows privilege escalation
Logs obtained from [the mordor dataset project](https://securitydatasets.com/notebooks/atomic/windows/privilege_escalation/SDWIN-190518200432.html)

In [3]:
def select_relevant_columns_prompt(adversary_tactic: str, df_description: str):
    prompt = f"""Based on the data below, determine which windows event field names are most valuable for detecting {adversary_tactic}?
    
Markdown table showing windows event field names and the count of unique values they contain:
{df_description}

"""
    
    prompt += "Format the selected field names as a list.\n"
    prompt += f"The following windows event field names are most valuable for detecting {adversary_tactic}:"
    return prompt

def brainstorm_filters_prompt(adversary_tactic: str, column_names: str):
    prompt = f"""pandas Dataframe column names:
{column_names}
"""

    prompt += f"""
- Create 10 diverse, and highly effective pandas DataFrame filters that find indications of {adversary_tactic}
- Use only the pandas Dataframe columns names listed above to create the filters.
- The filters are applied to a pandas DataFrame named 'df'. 
- The resulting pandas DataFrame is also named 'df'.

Filters:
    """
    return prompt

def format_filters_prompt(filters_response: str):
    prompt = f"""{filters_response}
    
Show only executable python code.
Show no numbers.
Show one python statement per line.
Python statements:
    """
    return prompt

TACTIC = "Windows privilege escalation"

In [4]:
df = json_pd.read_json(path_or_buf=PSINJECT, lines=True)
df.iloc[:3]

,@version,EventType,ThreadID,EventTime,Task,Channel,SourceName,Opcode,Hostname,@timestamp,...,param1,param2,MessageTotal,ScriptBlockText,ScriptBlockId,MessageNumber,Device,ShareLocalPath,ShareName,RelativeTargetName
0,1,INFO,0,2020-08-07 14:32:07,8,Windows PowerShell,PowerShell,Info,WORKSTATION5.theshire.local,2020-08-07T18:32:07.555Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,INFO,0,2020-08-07 14:32:07,8,Windows PowerShell,PowerShell,Info,WORKSTATION5.theshire.local,2020-08-07T18:32:07.555Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,INFO,7324,2020-08-07 14:32:07,106,Microsoft-Windows-PowerShell/Operational,Microsoft-Windows-PowerShell,To be used when operation is just executing a ...,WORKSTATION5.theshire.local,2020-08-07T18:32:07.556Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
df_description = df.describe(include="all").iloc[[0]].to_markdown()

Find most important column names

In [88]:
columns_prompt = select_relevant_columns_prompt(adversary_tactic=TACTIC, df_description=df_description)
print(columns_prompt)

Based on the data below, determine which windows event field names are most valuable for detecting Windows privilege escalation?
    
Markdown table showing windows event field names and the count of unique values they contain:
|       |   @version |   EventType |   ThreadID |   EventTime |   Task |   Channel |   SourceName |   Opcode |   Hostname |   @timestamp |   Message |   Category |   SourceModuleName |   EventReceivedTime |   port |   ExecutionProcessID |   host |   Severity |   SeverityValue |   EventID |   RecordNumber |   SourceModuleType |   Keywords |   tags |   Version |   AccountType |   OpcodeValue |   AccountName |   ContextInfo |   Payload |   Domain |   ActivityID |   UserID |   ProviderGuid |   SourcePort |   ProcessId |   Application |   LayerRTID |   LayerName |   SourceAddress |   FilterRTID |   Protocol |   EventTypeOrignal |   UtcTime |   TargetObject |   Image |   ProcessGuid |   RuleName |   DestAddress |   Direction |   RemoteMachineID |   DestPort |   Remote

In [89]:
columns_response = complete_prompt(columns_prompt, system_role=SYSTEM_ROLE, temperature = 0)
print(columns_response)

- AccountType
- AccountName
- Domain
- EventID
- EventType
- LogonType
- NewProcessName
- ParentProcessName
- ProcessId
- SourceImage
- TargetImage
- TargetProcessGUID
- TargetUserSid
- UserID


In [90]:
filters_prompt = brainstorm_filters_prompt(adversary_tactic=TACTIC, column_names=columns_response)
print(filters_prompt)

pandas Dataframe column names:
- AccountType
- AccountName
- Domain
- EventID
- EventType
- LogonType
- NewProcessName
- ParentProcessName
- ProcessId
- SourceImage
- TargetImage
- TargetProcessGUID
- TargetUserSid
- UserID

- Create 10 diverse, and highly effective pandas DataFrame filters that find indications of Windows privilege escalation
- Use only the pandas Dataframe columns names listed above to create the filters.
- The filters are applied to a pandas DataFrame named 'df'. 
- The resulting pandas DataFrame is also named 'df'.

Filters:
    


In [95]:
filters_response = complete_prompt(filters_prompt, system_role=SYSTEM_ROLE, temperature=0.12)
print(filters_response)

1. df = df[df['EventType'].str.contains('Privilege Escalation')]
2. df = df[df['LogonType'].isin(['3', '4', '5', '7', '8', '9', '10', '11'])]
3. df = df[df['NewProcessName'].str.contains('cmd.exe|powershell.exe', regex=True)]
4. df = df[df['ParentProcessName'].str.contains('explorer.exe', regex=True)]
5. df = df[df['ProcessId'].isin(['0', '4', '8', '12', '16', '20', '24', '28', '32', '36'])]
6. df = df[df['SourceImage'].str.contains('mimikatz.exe', regex=True)]
7. df = df[df['TargetImage'].str.contains('lsass.exe', regex=True)]
8. df = df[df['TargetProcessGUID'].notnull()]
9. df = df[df['TargetUserSid'].str.contains('S-1-5-21', regex=True)]
10. df = df[df['UserID'].str.contains('S-1-5-21', regex=True)]


In [96]:
format_prompt = format_filters_prompt(filters_response)
formatted_response = complete_prompt(format_prompt, system_role=SYSTEM_ROLE, temperature=0)
print(formatted_response)

df = df[df['EventType'].str.contains('Privilege Escalation')]
df = df[df['LogonType'].isin(['3', '4', '5', '7', '8', '9', '10', '11'])]
df = df[df['NewProcessName'].str.contains('cmd.exe|powershell.exe', regex=True)]
df = df[df['ParentProcessName'].str.contains('explorer.exe', regex=True)]
df = df[df['ProcessId'].isin(['0', '4', '8', '12', '16', '20', '24', '28', '32', '36'])]
df = df[df['SourceImage'].str.contains('mimikatz.exe', regex=True)]
df = df[df['TargetImage'].str.contains('lsass.exe', regex=True)]
df = df[df['TargetProcessGUID'].notnull()]
df = df[df['TargetUserSid'].str.contains('S-1-5-21', regex=True)]
df = df[df['UserID'].str.contains('S-1-5-21', regex=True)]


In [97]:
resulting_logs_size = []
dfs = []
for filter_command in formatted_response.split("\n"):
    filter_command = filter_command.replace("`","")
    filter_command = filter_command.replace('"',"")
    filter_command = filter_command.strip()
    df = json_pd.read_json(path_or_buf=PSINJECT, lines=True)
    df.fillna('', inplace=True)
    try:
        exec(filter_command)
        resulting_logs_size.append(df.shape[0])
        if 0 < df.shape[0] < 50:
            dfs.append(df)
    except:
        print("filter failed")
        resulting_logs_size.append(None)

In [98]:
resulting_logs_size

[0, 0, 0, 0, 5, 0, 2, 5898, 0, 2009]

In [99]:
len(dfs)

2

In [80]:
def format_messages_as_str(messages: List[str]):
    messages_str = ""
    for message in messages:
        message = message.strip().replace("\t", "").replace("\r\n", "")
        messages_str += f"- {message} \n"
    return messages_str

def get_summarize_prompt(messages: str, adversary_tactic: str):
    prompt = f"""You are analyzing windows event logs for signs of {adversary_tactic}.

The following event messages happen in chronological order:
{messages}

Are there indications of {adversary_tactic} apparent in the logs?
"""
    return prompt

In [100]:
df = pd.concat(dfs)
df

,@version,EventType,ThreadID,EventTime,Task,Channel,SourceName,Opcode,Hostname,@timestamp,...,param1,param2,MessageTotal,ScriptBlockText,ScriptBlockId,MessageNumber,Device,ShareLocalPath,ShareName,RelativeTargetName
4699,1,AUDIT_SUCCESS,96,2020-08-07 14:32:53,12810,Security,Microsoft-Windows-Security-Auditing,Info,MORDORDC.theshire.local,2020-08-07T18:33:08.046Z,...,,,,,,,,,,
4701,1,AUDIT_SUCCESS,96,2020-08-07 14:32:53,12810,Security,Microsoft-Windows-Security-Auditing,Info,MORDORDC.theshire.local,2020-08-07T18:33:08.046Z,...,,,,,,,,,,
4702,1,AUDIT_SUCCESS,96,2020-08-07 14:32:53,12810,Security,Microsoft-Windows-Security-Auditing,Info,MORDORDC.theshire.local,2020-08-07T18:33:08.046Z,...,,,,,,,,,,
4756,1,INFO,4204,2020-08-07 14:32:55,3,Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon,,MORDORDC.theshire.local,2020-08-07T18:33:09.830Z,...,,,,,,,,,,
4757,1,INFO,4204,2020-08-07 14:32:55,3,Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon,,MORDORDC.theshire.local,2020-08-07T18:33:09.830Z,...,,,,,,,,,,
2269,1,INFO,4364,2020-08-07 14:32:46,10,Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon,,WORKSTATION5.theshire.local,2020-08-07T18:32:49.308Z,...,,,,,,,,,,
2272,1,INFO,4364,2020-08-07 14:32:46,10,Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon,,WORKSTATION5.theshire.local,2020-08-07T18:32:49.310Z,...,,,,,,,,,,


In [101]:
messages = df["Message"].to_list()
messages_str = format_messages_as_str(messages)

In [102]:
prompt = get_summarize_prompt(messages_str, "Windows privilege escalation")
print(prompt)

You are analyzing windows event logs for signs of Windows privilege escalation.

The following event messages happen in chronological order:
- The Windows Filtering Platform has permitted a bind to a local port.Application Information:Process ID:4Application Name:SystemNetwork Information:Source Address:::Source Port:56259Protocol:6Filter Information:Filter Run-Time ID:0Layer Name:Resource AssignmentLayer Run-Time ID:38 
- The Windows Filtering Platform has permitted a connection.Application Information:Process ID:4Application Name:SystemNetwork Information:Direction:OutboundSource Address:fe80::3816:b2ee:1b9b:324bSource Port:56259Destination Address:fe80::3816:b2ee:1b9b:324bDestination Port:445Protocol:6Filter Information:Filter Run-Time ID:65853Layer Name:ConnectLayer Run-Time ID:50 
- The Windows Filtering Platform has permitted a connection.Application Information:Process ID:4Application Name:SystemNetwork Information:Direction:InboundSource Address:fe80::3816:b2ee:1b9b:324bSource 

In [103]:
print(complete_prompt(prompt, system_role=SYSTEM_ROLE, temperature=0))

Yes, there are indications of Windows privilege escalation in the logs. 

The first event message shows that a local port was bound to by a system process with Process ID 4. 

The second and third event messages show that an outbound and inbound connection was permitted respectively, between the same source and destination addresses and ports. 

The fourth and fifth event messages show that a network connection was detected, initiated by process ID 4 (which is a system process) and connecting to port 445, which is commonly used for SMB file sharing. 

The sixth and seventh event messages show that process ID 452 (which is a svchost.exe process) accessed the lsass.exe process with granted access of 0x1000, which is a common technique used for privilege escalation. 

Overall, the combination of these events suggests that an attacker may have used a system process to establish a connection to a remote system and then used a privilege escalation technique to access sensitive information or

# 2. Detect signs of lateral movement

In [150]:
TACTIC = "Windows lateral movement"
df = json_pd.read_json(path_or_buf=MIMIKATZ, lines=True)
df_description = df.describe().iloc[[0]].to_markdown()

In [151]:
columns_prompt = select_relevant_columns_prompt(adversary_tactic=TACTIC, df_description=df_description)
print(columns_prompt)
print()
columns_response = complete_prompt(columns_prompt, system_role=SYSTEM_ROLE, temperature = 0)
print(columns_response)

Based on the data below, determine which windows event field names are most valuable for detecting Windows lateral movement?
    
Markdown table showing windows event field names and the count of unique values they contain:
|       |   SeverityValue |   @version |   port |   Task |   Version |   RecordNumber |   ThreadID |   EventID |   Keywords |   ExecutionProcessID |   OpcodeValue |   FilterRTID |   LayerRTID |   SourcePort |   DestPort |   SourceThreadId |   KeyLength |   IpPort |   LogonType |   EventIdx |   EventCountTotal |   DestinationPort |   TerminalSessionId |   ParentProcessId |   RestrictedSidCount |   QueryStatus |   ERROR_EVT_UNRESOLVED |   MiniportNameLen |
|:------|----------------:|-----------:|-------:|-------:|----------:|---------------:|-----------:|----------:|-----------:|---------------------:|--------------:|-------------:|------------:|-------------:|-----------:|-----------------:|------------:|---------:|------------:|-----------:|------------------:|-----

In [152]:
filters_prompt = brainstorm_filters_prompt(adversary_tactic=TACTIC, column_names=columns_response)
filters_response = complete_prompt(filters_prompt, system_role=SYSTEM_ROLE, temperature=0.15)
print(filters_response)

1. Filter for any rows where the ParentProcessId is not null and the ExecutionProcessID is different from the ParentProcessId. This indicates a process was spawned by another process, which is a common technique used in lateral movement.

```
df = df[(df['ParentProcessId'].notnull()) & (df['ExecutionProcessID'] != df['ParentProcessId'])]
```

2. Filter for any rows where the ThreadID is not null and the IpPort is not null. This indicates network activity, which is often used in lateral movement.

```
df = df[(df['ThreadID'].notnull()) & (df['IpPort'].notnull())]
```

3. Filter for any rows where the SourcePort is not null and the DestPort is null. This indicates outbound network traffic, which is often used in lateral movement.

```
df = df[(df['SourcePort'].notnull()) & (df['DestPort'].isnull())]
```

4. Filter for any rows where the SourcePort is not null and the DestPort is not null and they are different. This indicates network traffic between two different ports, which is often us

In [153]:
format_prompt = format_filters_prompt(filters_response)
formatted_response = complete_prompt(format_prompt, system_role=SYSTEM_ROLE, temperature=0)
print(formatted_response)

df = df[(df['ParentProcessId'].notnull()) & (df['ExecutionProcessID'] != df['ParentProcessId'])]
df = df[(df['ThreadID'].notnull()) & (df['IpPort'].notnull())]
df = df[(df['SourcePort'].notnull()) & (df['DestPort'].isnull())]
df = df[(df['SourcePort'].notnull()) & (df['DestPort'].notnull()) & (df['SourcePort'] != df['DestPort'])]
df = df[(df['SourcePort'].notnull()) & (df['DestPort'].notnull()) & (df['SourcePort'] == df['DestPort'])]
df = df[(df['IpPort'].notnull()) & (df['DestPort'].notnull()) & (df['IpPort'] != df['DestPort'])]
df = df[(df['IpPort'].notnull()) & (df['DestPort'].notnull()) & (df['IpPort'] == df['DestPort'])]
df = df[(df['ThreadID'].notnull()) & (df['IpPort'].isnull())]
df = df[(df['ExecutionProcessID'].notnull()) & (df['ParentProcessId'].isnull())]
df = df[(df['ExecutionProcessID'].notnull()) & (df['ParentProcessId'].isnull()) & (df['ThreadID'].notnull())]


In [162]:
resulting_logs_size = []
dfs = []
for filter_command in formatted_response.split("\n"):
    filter_command = filter_command.replace("`","")
    filter_command = filter_command.replace('"',"")
    filter_command = filter_command.strip()
    df = json_pd.read_json(path_or_buf=MIMIKATZ, lines=True)
    df.fillna('', inplace=True)
    try:
        exec(filter_command)
        resulting_logs_size.append(df.shape[0])
        if 0 < df.shape[0] < 50:
            dfs.append(df)
    except:
        print("filter failed")
        resulting_logs_size.append(None)

In [163]:
resulting_logs_size

[790, 790, 0, 113, 677, 57, 733, 0, 0, 0]

In [169]:
df = json_pd.read_json(path_or_buf=MIMIKATZ, lines=True)
df = df[(df['SourcePort'].notnull()) & (df['DestPort'].notnull()) & (df['SourcePort'] != df['DestPort'])]
df.shape

(50, 181)

In [172]:
messages = df["Message"].iloc[:20].to_list()
messages_str = format_messages_as_str(messages)
prompt = get_summarize_prompt(messages_str, TACTIC)
print(prompt)

You are analyzing windows event logs for signs of Windows lateral movement.

The following event messages happen in chronological order:
- The Windows Filtering Platform has permitted a connection.Application Information:Process ID:3124Application Name:\device\harddiskvolume2\windowsazure\guestagent_2.7.41491.993_2020-09-16_193546\guestagent\windowsazureguestagent.exeNetwork Information:Direction:OutboundSource Address:172.18.39.6Source Port:62303Destination Address:168.63.129.16Destination Port:80Protocol:6Filter Information:Filter Run-Time ID:71768Layer Name:ConnectLayer Run-Time ID:48 
- The Windows Filtering Platform has permitted a connection.Application Information:Process ID:3692Application Name:\device\harddiskvolume2\windowsazure\guestagent_2.7.41491.993_2020-09-16_193257\guestagent\windowsazureguestagent.exeNetwork Information:Direction:OutboundSource Address:172.18.38.5Source Port:50092Destination Address:168.63.129.16Destination Port:80Protocol:6Filter Information:Filter Ru

In [173]:
print(complete_prompt(prompt, system_role=SYSTEM_ROLE, temperature=0))

Yes, there are indications of Windows lateral movement apparent in the logs. 

The logs show multiple outbound connections from different processes (identified by their process IDs) to the same destination IP address (168.63.129.16), which is a Microsoft Azure IP address. This could indicate that an attacker is attempting to move laterally within the network by using compromised credentials or exploiting vulnerabilities to gain access to other systems within the same network. 

Additionally, the connection to the IP address 169.254.169.254 is a well-known Azure metadata endpoint, which could be used by an attacker to obtain sensitive information about the Azure environment. 

Finally, the connection from the Mimikatz tool to the destination IP address 172.18.38.5 on port 135 is a common technique used by attackers to perform pass-the-hash attacks and gain access to other systems within the network.
